# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [3]:
# load data from database
engine = create_engine('sqlite:///TweetsDatabase.db')
df = pd.read_sql_table('TweetsDatabase', engine) 

In [4]:
# preview df
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
y = df.drop(['id','original','message','genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [6]:
# download necessary NLTK data
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# import statements
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oleksandryatsenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oleksandryatsenko/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oleksandryatsenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def tokenize(text):
    text = text.lower() # Convert to lowercase
    text = re.sub('[^A-Za-z0-9]+',' ', text) # removing punctuation
    words = nltk.word_tokenize(text) # tokenizing words
    stop_words = set(stopwords.words('english')) # set of stopwords
    filtered_words = [w for w in words if not w.lower() in stop_words]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in filtered_words:
        # lemmatization
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)
    return clean_tokens

### My experimenting with pipelines

In [8]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [9]:
# testing tokenize fucntion
X[0]

tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [10]:
# loop testing of tokenize fucntion
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



In [11]:
# before creating a pipeline let's test if we can achive key steps in a 
# simple main function. Later on we will do a proper refactoring with
# all the steps nested inside of a pipeline.

In [12]:
# managing size of random forest regressor
# https://mljar.com/blog/random-forest-memory/
# https://medium.com/all-things-ai/in-depth-parameter-tuning-for-random-forest-d67bb7e920d

In [13]:
# Extracting sklearn liabraries

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=56)
print(np.unique(y_train))

# https://github.com/ageron/handson-ml/issues/360 - look here

['0' '1' '2']


In [15]:
# trying to convert predictions to single - digits

In [16]:
X = df['message']
y = df.drop(['id','original','message','genre'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=56)

vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = RandomForestClassifier(n_estimators = 10, max_depth=6) # random forest consumes too much memory
#clf = MultiOutputClassifier(SGDClassifier(random_state=42), n_jobs=1)

# train classifier
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)
clf.fit(X_train_tfidf, y_train)

# predict on test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)


In [28]:
confusion_matrix(y_test, y_pred)

ValueError: multiclass-multioutput is not supported

In [39]:
print (y_pred)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [36]:
from sklearn.metrics import classification_report

print(y_test.shape)
print(y_pred.shape)

# 
y_test_arr = y_test.values

y_test_arg = np.argmax(y_test_arr, axis=1)
y_pred_arg = np.argmax(y_pred, axis=1)

# print(y_test_arg)
# print(y_test_arg)

# print(classification_report(y_test_arr, y_pred))

confusion_matrix(y_test_arg, y_pred_arg)

(18352, 36)
(18352, 36)


array([[18352]])

In [27]:
y_test_arr.shape

(18352, 36)

In [28]:
y_pred.shape

(18352, 36)

In [51]:


# reasons for error 
# https://stackoverflow.com/questions/20335853/scikit-multilabel-classification-valueerror-bad-input-shape

# STOPPED HERE
# converting one hot encoded multi-label back to actual labels 
# https://stackoverflow.com/questions/54589669/confusion-matrix-error-classification-metrics-cant-handle-a-mix-of-multilabel/54595455

(      related request offer aid_related medical_help medical_products  \
 10531       1       0     0           0            0                0   
 18648       1       0     0           1            0                1   
 7123        1       1     0           1            0                0   
 17123       1       0     0           0            0                0   
 24677       1       0     0           1            1                1   
 3477        1       1     0           1            0                0   
 1857        1       0     0           0            0                0   
 18532       1       0     0           1            0                0   
 10229       1       0     0           0            0                0   
 21470       1       0     0           1            0                0   
 2393        1       0     0           0            0                0   
 20809       1       1     0           0            0                0   
 23586       0       0     0          

In [52]:
y_pred

NameError: name 'y_pred' is not defined

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.